In [11]:
# torch svd as approach to solve

import torch
import torch.nn as nn
import scipy.optimize as opt
import numpy as np
import sys
sys.path.append("../")
from ddn.pytorch.node import *
import warnings
warnings.filterwarnings('ignore')

class NormalizedCuts(AbstractDeclarativeNode):
    """
    A declarative node to embed Normalized Cuts into a Neural Network
    
    Normalized Cuts and Image Segmentation https://people.eecs.berkeley.edu/~malik/papers/SM-ncut.pdf
    Shi, J., & Malik, J. (2000)
    """
    def __init__(self):
        super().__init__()
        
    def objective(self, x):
        # x is an NxN symmetrical matrix with W(i,j) = w_ij
        D = x.sum(1).diag() # D is an NxN diagonal matrix with d on diagonal, for d(i) = sum_j(w(i,j))
        ONE = torch.ones(x.size(dim=0),1)   # Nx1 vector of all ones
        L = D - x
        
        Lsym = torch.mm(torch.mm(torch.diag(torch.pow(torch.diag(D),-0.5)),L),torch.diag(torch.pow(torch.diag(D),-0.5)))
        return Lsym

    def solve(self, x):
        # SVD works for any shape input, torch.linalg.eig for square
        [u,s,v] = torch.svd(self.objective(x))
        y = u
        return y, _
    
torch.set_default_tensor_type(torch.DoubleTensor)

node = NormalizedCuts()
x = torch.tensor([[0,1,0], [2,0,3], [0,4,0]]).double()
y,_ = node.solve(x)
print(x)
print(y)
print("done")
# torch.cuda.set_device(gpu)
# model = model.cuda(gpu)

tensor([[0., 1., 0.],
        [2., 0., 3.],
        [0., 4., 0.]])
tensor([[-0.4040, -0.6937,  0.5963],
        [ 0.7306,  0.1475,  0.6667],
        [-0.5504,  0.7050,  0.4472]])
done


In [ ]:
import torch
from torchvision import transforms, datasets

# https://github.com/pytorch/vision/issues/2212#issuecomment-944225785
# Compose and StandardCompose don't work together nicely?

data_transform = Compose([
    transforms.Resize(size=(100,100)),
    transforms.Grayscale(1),
    transforms.ToTensor()
    ])
    
train_city = torchvision.datasets.Cityscapes(root='./data/cityscapes/', split='train', mode='fine',
                     target_type='semantic', transforms=data_transform)

test_city = torchvision.datasets.Cityscapes(root='./data/cityscapes/', split='test', mode='fine',
                     target_type='semantic', transforms=data_transform)

train_loader = torch.utils.data.DataLoader(train_city, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_city, batch_size=1, shuffle=True)

# visualise
train_img = next(iter(train_loader))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        """
        instatiate parameter
        """
        super().__init__()
    
# GPU ID to use
gpu = 1

# Create model, set to use GPU
model = Net()